In [ ]:
# 라이브러리 임포트
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import pandas, numpy, random
import matplotlib.pyplot as plt
# 데이터 로드
# MNIST dataset
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
USE_CUDA = torch.cuda.is_available() # GPU를 사용가능하면 True, 아니라면 False를 리턴
device = torch.device("cuda" if USE_CUDA else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용
print("다음 기기로 학습합니다:", device)

mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
data_loader = DataLoader(dataset=mnist_train,
                                          batch_size=1, # 배치 크기는 100
                                          shuffle=True,
                                          drop_last=True)

# 동일한 임의 데이터를 생성하기위한 함수
def generate_random_image(size):
    random_data = torch.rand(size)
    return random_data

def generate_random_seed(size):
    random_data = torch.randn(size)
    return random_data

In [ ]:
# 분류기 클래스
class Discriminator(nn.Module):
    def __init__(self):
        # 부모 클래스 초기화
        super().__init__()
        # 신경망 레이어 정의
        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.LeakyReLU(0.02),
            nn.LayerNorm(200),
            nn.Linear(200, 1),
            nn.Sigmoid()
        )
        # 손실함수 설정
        self.loss_function = nn.BCELoss()
        # 옵티마이저 설정
        self.optimiser = torch.optim.Adam(self.parameters(), lr=0.0001)
        # 변수 초기화
        self.counter = 0;
        self.progress = []
    def forward(self, inputs):
        # 모델 실행
        return self.model(inputs.to(device))
    def train(self, inputs, targets):
        # 신경망의 결과 계산
        outputs = self.forward(inputs).to(device)
        # 손실 계산
        loss = self.loss_function(outputs, targets)
        # 카운터를 증가시키고 10회마다 오차 저장
        self.counter += 1;
        if (self.counter % 10 == 0):
            self.progress.append(loss.item())
        if (self.counter % 10000 == 0):
            print("counter = ", self.counter)
        # 기울기 초기화, 역전파 실행, 가중치 갱신
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()
    def plot_progress(self):
        df = pandas.DataFrame(self.progress, columns=['loss'])
        df.plot(ylim=(0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5, 1.0, 5.0))

In [ ]:
# 생성기 class
class Generator(nn.Module):
    def __init__(self):
        # 파이토치 부모 클래스 초기화
        super().__init__()
        # 신경망 레이어 정의
        self.model = nn.Sequential(
            nn.Linear(100, 200),
            nn.LeakyReLU(0.02),
            nn.LayerNorm(200),
            nn.Linear(200, 784),
            nn.Sigmoid()
        )
        # 옵티마이저 생성
        self.optimiser = torch.optim.Adam(self.parameters(), lr=0.0001)
        # 진행 측정을 위한 변수 초기화
        self.counter = 0;
        self.progress = []
    def forward(self, inputs):        
        # 모델 실행
        return self.model(inputs.to(device))
    def train(self, D, inputs, targets):
        # 신경망 출력 계산
        g_output = self.forward(inputs).to(device)
        # 판별기에 값 전달
        d_output = D.forward(g_output).to(device)
        # 오차 계산
        loss = D.loss_function(d_output, targets)
        # 매 10회마다 에러를 누적하고 카운터를 증가
        self.counter += 1;
        if (self.counter % 10 == 0):
            self.progress.append(loss.item())
        # 기울기를 초기화 하고 역전파 후 가중치 갱신
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()
    def plot_progress(self):
        df = pandas.DataFrame(self.progress, columns=['loss'])
        df.plot(ylim=(0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5, 1.0, 5.0))

In [ ]:
%%time 
# 판별기 및 생성기 생성
D = Discriminator().to(device)
G = Generator().to(device)
epochs = 4
for epoch in range(epochs):
    print ("epoch = ", epoch + 1)
    for image_data_tensor, target_tensor in data_loader:
        # 참일 경우 판별기 훈련
        D.train(image_data_tensor.view(784).to(device), torch.FloatTensor([1.0]).to(device))
        # 거짓일 경우 판별기 훈련
        # G의 기울기가 계산되지 않도록 detach() 함수를 이용
        D.train(G.forward(generate_random_seed(100)).detach().to(device), torch.FloatTensor([0.0]).to(device))
        # 생성기 훈련
        G.train(D, generate_random_seed(100).to(device), torch.FloatTensor([1.0]).to(device))

In [ ]:
# 판별기 오차 플롯

D.plot_progress()

In [ ]:
# 생성기 오차 플롯

G.plot_progress()

In [ ]:
# 훈련된 생성기로부터 몇개의 출력을 플롯
# plot a 3 column, 2 row array of generated images
f, axarr = plt.subplots(2,3, figsize=(16,8))
for i in range(2):
    for j in range(3):
        output = G.forward(generate_random_seed(100))
        img = output.detach().numpy().reshape(28,28)
        axarr[i,j].imshow(img, interpolation='none', cmap='Blues')